In [ ]:
#!git clone https://github.com/SinaPourSoltani/Visuomotor-Behaviour-Cloning.git

In [2]:
#import sys
#sys.path.append('Visuomotor-Behaviour-Cloning')

from BehaviourCloningCNN import *

ModuleNotFoundError: No module named 'pandas'

In [ ]:
data_link = "https://github.com/SinaPourSoltani/Visuomotor-Behaviour-Cloning/releases/download/v0.1/images.zip"

load_data(data_link)
train_loader, valid_loader, test_loader = get_data_loaders(*get_episodes())
model = get_model()

In [ ]:
plot_history(*train(model,train_loader,valid_loader, lr=1e-4))

In [ ]:
test_acc = one_epoch(model, test_loader)[1]
print(f'{test_acc * 100:.1f} % test accuracy')

In [ ]:
filename = "ResNet18_Neps" + str(len(get_episodes()[0])) + ".pth"
torch.save(model.state_dict(), filename)